# Preparing data frame of case counts and variant frequencies

## Setup

### Imports

In [ ]:
import pandas as pd

### Data

Download Nextstrain-curated metadata TSV of GenBank database, select a subset of columns, save for later use, and analyze.

In [ ]:
path_to_metadata = "https://data.nextstrain.org/files/ncov/open/metadata.tsv.gz"

In [ ]:
path_to_cases = "https://data.cdc.gov/api/views/9mfq-cb36/rows.csv?accessType=DOWNLOAD"

In [ ]:
output_sequences = "variants-us_location-variant-sequence-counts.python.tsv"

In [ ]:
output_cases = "variants-us_location-case-counts.python.tsv"

In [ ]:
metadata = pd.read_csv(
    path_to_metadata,
    sep="\t",
    usecols=(
        "strain",
        "date",
        "country",
        "division",
        "Nextstrain_clade"
    ),
    parse_dates=["date"],
)

metadata.to_csv(
    "open_metadata_pruned.tsv",
    sep="\t",
    index=False,
)

In [ ]:
metadata.shape

In [ ]:
metadata.head()

### Thresholds

Define thresholds for clades and states.

In [ ]:
min_records_per_clade = 2000

In [ ]:
min_records_per_division = 5000

### Clade definitions

In [ ]:
clade_to_variant = {
    "20H (Beta, V2)": "Beta",
    "20I (Alpha, V1)": "Alpha",
    "20J (Gamma, V3)": "Gamma",
    "21A (Delta)": "Delta",
    "21C (Epsilon)": "Epsilon",
    "21F (Iota)": "Iota",
    "21H (Mu)": "Mu",
    "21I (Delta)": "Delta",
    "21J (Delta)": "Delta",
}

In [ ]:
variants = {
    "Alpha",
    "Beta",
    "Gamma",
    "Delta",
    "Epsilon",
    "Iota", 
    "Mu",
}

### Dates

In [ ]:
start_date_for_cases = "2020-11-01"

In [ ]:
start_date = "2021-01-01"

In [ ]:
end_date = "2021-10-01"

In [ ]:
dates = pd.date_range(
    start_date,
    end_date,
    freq="D",
)

In [ ]:
dates

### States

Map full names of US states to abbreviations and vice versa. Data originally from https://raw.githubusercontent.com/jasonong/List-of-US-States/master/states.csv.

In [ ]:
states = [
    {'name': 'Alabama', 'abbreviation': 'AL'},
    {'name': 'Alaska', 'abbreviation': 'AK'},
    {'name': 'Arizona', 'abbreviation': 'AZ'},
    {'name': 'Arkansas', 'abbreviation': 'AR'},
    {'name': 'California', 'abbreviation': 'CA'},
    {'name': 'Colorado', 'abbreviation': 'CO'},
    {'name': 'Connecticut', 'abbreviation': 'CT'},
    {'name': 'Delaware', 'abbreviation': 'DE'},
    {'name': 'District of Columbia', 'abbreviation': 'DC'},
    {'name': 'Washington DC', 'abbreviation': 'DC'},
    {'name': 'Florida', 'abbreviation': 'FL'},
    {'name': 'Georgia', 'abbreviation': 'GA'},
    {'name': 'Hawaii', 'abbreviation': 'HI'},
    {'name': 'Idaho', 'abbreviation': 'ID'},
    {'name': 'Illinois', 'abbreviation': 'IL'},
    {'name': 'Indiana', 'abbreviation': 'IN'},
    {'name': 'Iowa', 'abbreviation': 'IA'},
    {'name': 'Kansas', 'abbreviation': 'KS'},
    {'name': 'Kentucky', 'abbreviation': 'KY'},
    {'name': 'Louisiana', 'abbreviation': 'LA'},
    {'name': 'Maine', 'abbreviation': 'ME'},
    {'name': 'Montana', 'abbreviation': 'MT'},
    {'name': 'Nebraska', 'abbreviation': 'NE'},
    {'name': 'Nevada', 'abbreviation': 'NV'},
    {'name': 'New Hampshire', 'abbreviation': 'NH'},
    {'name': 'New Jersey', 'abbreviation': 'NJ'},
    {'name': 'New Mexico', 'abbreviation': 'NM'},
    {'name': 'New York', 'abbreviation': 'NYC'},
    {'name': 'New York', 'abbreviation': 'NY'},
    {'name': 'North Carolina', 'abbreviation': 'NC'},
    {'name': 'North Dakota', 'abbreviation': 'ND'},
    {'name': 'Ohio', 'abbreviation': 'OH'},
    {'name': 'Oklahoma', 'abbreviation': 'OK'},
    {'name': 'Oregon', 'abbreviation': 'OR'},
    {'name': 'Maryland', 'abbreviation': 'MD'},
    {'name': 'Massachusetts', 'abbreviation': 'MA'},
    {'name': 'Michigan', 'abbreviation': 'MI'},
    {'name': 'Minnesota', 'abbreviation': 'MN'},
    {'name': 'Mississippi', 'abbreviation': 'MS'},
    {'name': 'Missouri', 'abbreviation': 'MO'},
    {'name': 'Pennsylvania', 'abbreviation': 'PA'},
    {'name': 'Rhode Island', 'abbreviation': 'RI'},
    {'name': 'South Carolina', 'abbreviation': 'SC'},
    {'name': 'South Dakota', 'abbreviation': 'SD'},
    {'name': 'Tennessee', 'abbreviation': 'TN'},
    {'name': 'Texas', 'abbreviation': 'TX'},
    {'name': 'Utah', 'abbreviation': 'UT'},
    {'name': 'Vermont', 'abbreviation': 'VT'},
    {'name': 'Virginia', 'abbreviation': 'VA'},
    {'name': 'Washington', 'abbreviation': 'WA'},
    {'name': 'West Virginia', 'abbreviation': 'WV'},
    {'name': 'Wisconsin', 'abbreviation': 'WI'},
    {'name': 'Wyoming', 'abbreviation': 'WY'},
    {'name': 'American Samoa', 'abbreviation': 'AS'},
    {'name': 'Federated States of Micronesia', 'abbreviation': 'FSM'},
    {'name': 'Guam', 'abbreviation': 'GU'},
    {'name': 'Northern Mariana Islands', 'abbreviation': 'MP'},
    {'name': 'Commonwealth of the Northern Mariana Islands', 'abbreviation': 'MP'},
    {'name': 'Puerto Rico', 'abbreviation': 'PR'},
    {'name': 'Republic of Palau', 'abbreviation': 'PW'},
    {'name': 'Republic of the Marshall Islands', 'abbreviation': 'RMI'},
    {'name': 'Virgin Islands', 'abbreviation': 'VI'},
    {'name': 'U.S. Virgin Islands', 'abbreviation': 'VI'},
]

In [ ]:
full_state_names_to_abbreviations = {
    state["name"]: state["abbreviation"]
    for state in states
}

In [ ]:
full_state_names_to_abbreviations

In [ ]:
abbreviations_to_full_state_names = {
    state["abbreviation"]: state["name"]
    for state in states
}

In [ ]:
abbreviations_to_full_state_names

## Sequence metadata

In [ ]:
metadata

In [ ]:
metadata.shape

Subset to recent samples, dropping records with ambiguous dates ("?" or "2021-06", etc.).

In [ ]:
unambiguous_dates = (metadata["date"] != "?") & (metadata["date"].str.count("-") == 2)

In [ ]:
unambiguous_dates.sum()

In [ ]:
date_since_start_date = (metadata["date"] >= start_date)

In [ ]:
date_since_start_date.sum()

In [ ]:
metadata = metadata[(date_since_start_date) & (unambiguous_dates)].copy()

In [ ]:
metadata.shape

Convert date strings to date types for easier operations.

In [ ]:
metadata["date"] = pd.to_datetime(metadata["date"])

Subset to older samples.

In [ ]:
date_cutoff = pd.to_datetime(end_date) + pd.DateOffset(days=1)

In [ ]:
date_before_cutoff_date = (metadata["date"] < date_cutoff)

In [ ]:
date_before_cutoff_date.sum()

In [ ]:
metadata = metadata[date_before_cutoff_date].copy()

Subset to the USA.

In [ ]:
metadata = metadata.query("country == 'USA'").copy()

In [ ]:
metadata.shape

Remove unassigned samples.

In [ ]:
metadata = metadata[~pd.isnull(metadata["Nextstrain_clade"])].copy()

In [ ]:
metadata.shape

## Variants to analyze

### Clades with data

Select all clades with a minimum number of metadata records for downstream analysis.

In [ ]:
clade_tallies = metadata["Nextstrain_clade"].value_counts()

In [ ]:
clade_tallies

In [ ]:
clade_tallies.shape

In [ ]:
clades_to_analyze = clade_tallies[clade_tallies > min_records_per_clade].index.tolist()

In [ ]:
clades_to_analyze

In [ ]:
# TODO: Define a list of Nextstrain clade names to filter by instead of searching for parentheses.
clades_to_analyze = sorted([
    clade
    for clade in clades_to_analyze
    if "(" in clade
])

In [ ]:
clades_to_analyze

### Mapping to variants

In [ ]:
metadata["variant"] = metadata["Nextstrain_clade"].map(clade_to_variant).fillna("other").astype(str)

In [ ]:
metadata["variant"].value_counts()

In [ ]:
metadata["variant"]

## States to analyze

Keep states with at least a minimum number of metadata records.

In [ ]:
state_tallies = metadata["division"].value_counts()

In [ ]:
state_tallies

In addition to getting states with enough records, drop records with a state value of "USA".

In [ ]:
states_to_analyze = sorted(state_tallies[(state_tallies > min_records_per_division)].drop("USA").index.tolist())

In [ ]:
states_to_analyze

In [ ]:
len(states_to_analyze)

Filter data to records for states with enough sequences.

In [ ]:
metadata = metadata[metadata["division"].isin(states_to_analyze)].copy()

In [ ]:
metadata.shape

## Export data frame of variant frequencies

Provision counts by date, state, and variant.

In [ ]:
metadata.head()

In [ ]:
counts_by_date_state_variant = metadata.groupby(
    [
        "date",
        "division",
        "variant",
    ],
    observed=True,
    as_index=False,
)["strain"].count().rename(
    columns={
        "strain": "sequences",
        "division": "location",
    }
).sort_values(["location", "variant", "date"])

In [ ]:
counts_by_date_state_variant.head()

In [ ]:
counts_by_date_state_variant.shape

In [ ]:
counts_by_date_state_variant.to_csv(
    output_sequences,
    sep="\t",
    index=False,
)

## State-level case data

Download with https://data.cdc.gov/api/views/9mfq-cb36/rows.csv?accessType=DOWNLOAD

### Data

In [ ]:
cases = pd.read_csv(
    path_to_cases,
    parse_dates=["submission_date"],
    usecols=[
        "submission_date",
        "state",
        "new_case",
        "new_death",
    ],
).sort_values([
    "submission_date",
    "state",
])

In [ ]:
cases.shape

In [ ]:
cases.head()

Filter cases to minimum start date.

In [ ]:
cases = cases[cases["submission_date"] >= start_date_for_cases].copy()

Drop any records with missing "new case" or "new death" values.

In [ ]:
cases = cases[(~pd.isnull(cases["new_case"])) & (~pd.isnull(cases["new_death"]))].copy()

Replace negative new case values with zeros.

In [ ]:
cases.loc[cases["new_case"] < 0, "new_case"] = 0

In [ ]:
cases.head()

In [ ]:
cases.shape

## Export data frame of case counts

Filter to cases between start and end date.

In [ ]:
cases = cases[(cases["submission_date"] >= start_date) & (cases["submission_date"] <= end_date)].copy()

Annotate full names for states.

In [ ]:
cases["location"] = cases["state"].map(abbreviations_to_full_state_names)

Confirm that none of the states have missing valus (indicating missing information in the abbreviation-to-name mapping).

In [ ]:
assert cases["location"].isnull().sum() == 0

Filter cases to states for analysis, based on genomic data above.

In [ ]:
cases = cases[cases["location"].isin(states_to_analyze)].reset_index()

In [ ]:
cases.shape

Sum cases across all states and dates, accounting for states/divisions with more than one abbreviation in the case data (e.g., "NYC" and "NY" for "New York").

In [ ]:
total_cases = cases.groupby(
    [
        "location",
        "submission_date"
    ],
    as_index=False,
)["new_case"].sum().rename(columns={
    "submission_date": "date",
    "new_case": "case",
})

In [ ]:
total_cases.shape

In [ ]:
total_cases.head()

In [ ]:
total_cases.to_csv(
    output_cases,
    sep="\t",
    index=False,
)